# Mini Project- IST 652

### House Prices: Advanced Regression Techniques

In [1]:
# Install Kaggle if you do not already have it.
# !pip install kaggle

In [2]:
# Import all files into the current directory using Kaggle API Key
# This command also requires a kaggle json file download as well from your kaggle acccount

# !kaggle competitions download -c house-prices-advanced-regression-techniques

In [3]:

# Also, lets import the packages that I would want to use to work with this type of data.

import numpy as np
import pandas as pd


In [4]:
# Now, the files are on your local directory, you can import the csv file to begin preprocessing 
# the data

housing_data_train = pd.read_csv('train.csv')
# print(housing_data_train.head(5))

housing_data_test = pd.read_csv("test.csv")
# print(housing_data_test.head(5))

# dataframe.size 
original_size_train = housing_data_train.size 
original_size_test = housing_data_test.size

# dataframe.shape 
original_shape_train = housing_data_train.shape 
original_shape_test = housing_data_test.shape

print("The size of the training data is", original_size_train)
print("The shape of the training data is", original_shape_train,"\n")


print("The size of the test data is", original_size_test)
print("The shape of the test data is", original_shape_test,"\n")

# Combine the train and test frame

frames = [housing_data_train, housing_data_test]

housing_data = pd.concat(frames, sort=True)

# dataframe.size 
combined_data_size = housing_data.size 


# dataframe.shape 
combined_data_shape = housing_data.shape 


print("The size of the combined data is", combined_data_size)
print("The shape of the combined data is", combined_data_shape)

The size of the training data is 118260
The shape of the training data is (1460, 81) 

The size of the test data is 116720
The shape of the test data is (1459, 80) 

The size of the combined data is 236439
The shape of the combined data is (2919, 81)


In [5]:
# Get column names
column_names = housing_data.columns
print(column_names)

# Get column data types
housing_data.dtypes

# Also check if the column is unique
for i in column_names:
    print('{} is unique: {}'.format(i, housing_data[i].is_unique))

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

Good! Our data has been quickly checked, we know the data types, and Id is a uniqie.

In [6]:
# Now we can make the index value "Id" since we know it is unique

housing_data = housing_data.set_index("Id")


In [7]:
# Lets do some explaratory data analysis to see what the data look like

# Check the number of NaN values in the dataframe

missing_data_NaN = housing_data.isnull().sum().sum()
print("The number of missing NaN types in the data frame are", missing_data_NaN)



The number of missing NaN types in the data frame are 15424


We are going to not use columns that are missing data of more than 80 percent


In [8]:
# Create a dataframe of the housing values that having missing data and the percentage amounts
percent_miss = housing_data.isnull().sum()/ len(housing_data)

df2 = pd.DataFrame(percent_miss)
df2 = df2.rename(columns={0: "Percent of missing values"})


df1 = pd.DataFrame(housing_data.isnull().sum())
df1 = df1.rename(columns={0: "Number of missing values"})

df = pd.concat([df1, df2], axis=1, sort=False)

sorted_df = df.sort_values(by =["Number of missing values"],ascending = False)
sorted_df.head(10)

,Number of missing values,Percent of missing values
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
SalePrice,1459,0.499829
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
GarageQual,159,0.054471
GarageFinish,159,0.054471
GarageCond,159,0.054471


We are going to not use columns that are missing data of more than 80 percent


In [9]:
# Put the columns that will be dropped into a list
columns_to_drop= ["PoolQC","MiscFeature","Alley","Fence"]

# Drop unwanted columns in the training data
housing_data.drop(columns_to_drop, inplace=True, axis=1)



In [10]:
# Check to see if the columns were dropped
housing_data.shape[1]

76

Lets conduct some more preprocessing on the rows that have missing values


In [11]:
# Another look at handling missing values
missing_info = list(housing_data.columns[housing_data.isnull().any()])

missing_info

['BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'Electrical',
 'Exterior1st',
 'Exterior2nd',
 'FireplaceQu',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'KitchenQual',
 'LotFrontage',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'SalePrice',
 'SaleType',
 'TotalBsmtSF',
 'Utilities']

In [12]:
for col in missing_info:
    percent_missing = housing_data[housing_data[col].isnull() == True].shape[0] / housing_data.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

percent missing for column BsmtCond: 0.028091812264474134
percent missing for column BsmtExposure: 0.028091812264474134
percent missing for column BsmtFinSF1: 0.00034258307639602604
percent missing for column BsmtFinSF2: 0.00034258307639602604
percent missing for column BsmtFinType1: 0.027064063035286058
percent missing for column BsmtFinType2: 0.027406646111682084
percent missing for column BsmtFullBath: 0.0006851661527920521
percent missing for column BsmtHalfBath: 0.0006851661527920521
percent missing for column BsmtQual: 0.02774922918807811
percent missing for column BsmtUnfSF: 0.00034258307639602604
percent missing for column Electrical: 0.00034258307639602604
percent missing for column Exterior1st: 0.00034258307639602604
percent missing for column Exterior2nd: 0.00034258307639602604
percent missing for column FireplaceQu: 0.48646796848235696
percent missing for column Functional: 0.0006851661527920521
percent missing for column GarageArea: 0.00034258307639602604
percent missing f

Now we have less columns with missing data, so lets explore these columns and determine a method for filling in missing values.

In [13]:
# Numerical columns summary statistics
housing_data.describe().T


,count,mean,std,min,25%,50%,75%,max
1stFlrSF,2919.0,1159.581706,392.362079,334.0,876.0,1082.0,1387.5,5095.0
2ndFlrSF,2919.0,336.483727,428.701456,0.0,0.0,0.0,704.0,2065.0
3SsnPorch,2919.0,2.602261,25.188169,0.0,0.0,0.0,0.0,508.0
BedroomAbvGr,2919.0,2.860226,0.822693,0.0,2.0,3.0,3.0,8.0
BsmtFinSF1,2918.0,441.423235,455.610826,0.0,0.0,368.5,733.0,5644.0
BsmtFinSF2,2918.0,49.582248,169.205611,0.0,0.0,0.0,0.0,1526.0
BsmtFullBath,2917.0,0.429894,0.524736,0.0,0.0,0.0,1.0,3.0
BsmtHalfBath,2917.0,0.061364,0.245687,0.0,0.0,0.0,0.0,2.0
BsmtUnfSF,2918.0,560.772104,439.543659,0.0,220.0,467.0,805.5,2336.0
EnclosedPorch,2919.0,23.098321,64.244246,0.0,0.0,0.0,0.0,1012.0


In [14]:
# Create a summary statistics for categorical columns that have missing

cols_obj = [x for x in housing_data.columns.to_list() if str(housing_data[x].dtype)=="object"]

cols_obj

housing_data[cols_obj].describe().T




,count,unique,top,freq
BldgType,2919,5,1Fam,2425
BsmtCond,2837,4,TA,2606
BsmtExposure,2837,4,No,1904
BsmtFinType1,2840,6,Unf,851
BsmtFinType2,2839,6,Unf,2493
BsmtQual,2838,4,TA,1283
CentralAir,2919,2,Y,2723
Condition1,2919,9,Norm,2511
Condition2,2919,8,Norm,2889
Electrical,2918,5,SBrkr,2671


From this view, we can see that our data has categorical and numerical data

In [15]:
# The number of numerical columns in our dataset (exclusing Id column)
print("There are",housing_data.shape[1]- len(cols_obj),"numerical columns in our dataset.")
# print(colcount.shape[1])

# The number of categorical columns in our dataset
print("There are",len(cols_obj),"in categorical columns in our dataset.")

There are 37 numerical columns in our dataset.
There are 39 in categorical columns in our dataset.


### Before we do furhter data preprocessing, lets gain insight into the data. 

#### Question 1: Compute the average sale price for houses that have 3 bedrooms and 2 full baths

BedroomAbvGr is the column that gives us the number of bedrooms in a house (does not include basement bedrooms)

SalePrice is the columns that gives us the price of the house

FullBath is the column that gives us the number of full bathrooms in a house (does not include basement bedroom)

In [31]:
# You can do an aggregate function using two groups in one function

housing_data.groupby(["FullBath","BedroomAbvGr"]).aggregate({"SalePrice":'mean'})

# Answer: $213,687.846501


SalePrice
FullBath BedroomAbvGr               
0        0             208591.800000
         1              48155.500000
         2             173769.000000
1        0             286000.000000
         1             165559.809524
         2             119790.633803
         3             138216.201705
         4             150849.081081
         5             120000.000000
2        1             268049.666667
         2             215187.545455
         3             213687.846501
         4             212340.128205
         5             184683.333333
         6             143779.000000
         8             200000.000000
3        3             250433.888889
         4             412162.550000
         5             245250.000000
         6                       NaN
4        4                       NaN
         6                       NaN

In [ ]:
q1 = housing_data.groupby(["FullBath","BedroomAbvGr"]).aggregate({"SalePrice":'mean'})

q1.to_csv("q1.csv")


#### Question 2: What is the median sale price of a house for each year?

In [17]:
housing_data.groupby(["YrSold"]).aggregate({"SalePrice":'median' }).T

YrSold,2006,2007,2008,2009,2010
SalePrice,163995.0,167000.0,164000.0,162000.0,155000.0


In [18]:
q2 = housing_data.groupby(["YrSold"]).aggregate({"SalePrice":'median' }).T
q2.to_csv("q2.csv")

### Now, lets get back to doing some exploring the data.

#### What can be some factors to influence the sale price of a house? 

For this example lets compare the condition of the  of the house listed at ExterCond compared to similiar types of houses. We are going to list the mean price across the different categories.

In [19]:
# You can do an aggregate function using two groups in one function

housing_data.groupby(["ExterCond"]).aggregate({"SalePrice":'mean', "SalePrice":'median'}).T

ExterCond,Ex,Fa,Gd,Po,TA
SalePrice,161000.0,95750.0,151250.0,76500.0,167370.0


In [20]:
housing_data.groupby(["BedroomAbvGr"]).aggregate({"SalePrice":'mean',"SalePrice":'median',}).T

BedroomAbvGr,0,1,2,3,4,5,6,8
SalePrice,202500.0,145250.0,137250.0,169945.0,193500.0,161500.0,141000.0,200000.0


In [21]:
housing_data.groupby(["FullBath"]).aggregate({"SalePrice":'mean',"SalePrice":'median', }).T

FullBath,0,1,2,3,4
SalePrice,145000.0,132375.0,196750.0,320000.0,NaN


#### How are we going to address columns that have missing data?

To handle both types of columns we can use the Impute class which is a part the sklearn package.
This will fill in the missing values by the most frequent observations for that class.

There are defintely more robust ways of handling missing data points, 
that include, creating interactin variables, filling in using the mean of column, and others.
I choose this method to handle both int and string types.

In [33]:
# Stopping Point

In [23]:
# After imputing the data merge the sales column back to the dataframe
housing_data.tail(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
2915,546,546,0,3,Twnhs,TA,No,0.0,0.0,Unf,...,WD,0,Pave,5,546.0,AllPub,0,1970,1970,2006
2916,546,546,0,3,TwnhsE,TA,No,252.0,0.0,Rec,...,WD,0,Pave,6,546.0,AllPub,0,1970,1970,2006
2917,1224,0,0,4,1Fam,TA,No,1224.0,0.0,ALQ,...,WD,0,Pave,7,1224.0,AllPub,474,1960,1996,2006
2918,970,0,0,3,1Fam,TA,Av,337.0,0.0,GLQ,...,WD,0,Pave,6,912.0,AllPub,80,1992,1992,2006
2919,996,1004,0,3,1Fam,TA,Av,758.0,0.0,LwQ,...,WD,0,Pave,9,996.0,AllPub,190,1993,1994,2006


In [24]:
# Put the target (SalePrice) in another DataFrame
target = pd.DataFrame(housing_data, columns=["SalePrice"]) 

# define the data/predictors as the pre-set feature names  
features =  housing_data.drop(['SalePrice'], axis=1)



In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(features,target,train_size=1460,random_state=3, shuffle= False)
print (len(X_test), len(y_test))

1459 1459


In [26]:
# We can use the sklearn Imputer to handle filling in for missing values.

from sklearn.impute import SimpleImputer

fill_NaN =SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed_DF_train = pd.DataFrame(fill_NaN.fit_transform(X_train)) # fills in all missing values
imputed_DF_train.columns = X_train.columns # reassigns the column names back
imputed_DF_train.index = X_train.index # reassigns an index to each row


In [27]:
# Lets take a look at values in the columns 
imputed_DF_train.head(5)


,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,856,854,0,3,1Fam,TA,No,706,0,GLQ,...,WD,0,Pave,8,856,AllPub,0,2003,2003,2008
2,1262,0,0,3,1Fam,TA,Gd,978,0,ALQ,...,WD,0,Pave,6,1262,AllPub,298,1976,1976,2007
3,920,866,0,3,1Fam,TA,Mn,486,0,GLQ,...,WD,0,Pave,6,920,AllPub,0,2001,2002,2008
4,961,756,0,3,1Fam,Gd,No,216,0,ALQ,...,WD,0,Pave,7,756,AllPub,0,1915,1970,2006
5,1145,1053,0,4,1Fam,TA,Av,655,0,GLQ,...,WD,0,Pave,9,1145,AllPub,192,2000,2000,2008


In [32]:
# Lets recheck our dataframe to see if there any more missing values
missing_data_NaN = imputed_DF_train.isnull().sum().sum()
print("The number of missing NaN types in the data frame are", missing_data_NaN)



The number of missing NaN types in the data frame are 0


In [ ]:

# Encode categorical data
cleaned_housing_data = pd.get_dummies(imputed_DF, columns = cols_obj)


In [ ]:
# Put the target (housing value -- MEDV) in another DataFrame
target = pd.DataFrame(cleaned_housing_data, columns=["SalePrice"]) 

# define the data/predictors as the pre-set feature names  
features =  cleaned_housing_data.drop(['SalePrice'], axis=1)


In [ ]:
## Without a constant

import statsmodels.api as sm

X = features
y = target["SalePrice"]


# Note the difference in argument order
#model = sm.OLS(y, X).fit()
model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()


In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

X_train,X_test,y_train,y_test=train_test_split(features,target,train_size=1460,random_state=3, shuffle= False)
print (len(X_test), len(y_test))
lr = LinearRegression()
lr.fit(X_train, y_train)
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
rr100 = Ridge(alpha=100) #  comparison with alpha value
rr100.fit(X_train, y_train)
train_score=lr.score(X_train, y_train)
test_score=lr.score(X_test, y_test)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
Ridge_train_score100 = rr100.score(X_train,y_train)
Ridge_test_score100 = rr100.score(X_test, y_test)
print ("linear regression train score:", train_score)
print ("linear regression test score:", test_score)
print ("ridge regression train score low alpha:", Ridge_train_score)
print ("ridge regression test score low alpha:", Ridge_test_score)
print ("ridge regression train score high alpha:", Ridge_train_score100)
print ("ridge regression test score high alpha:", Ridge_test_score100)



In [ ]:
# difference of lasso and ridge regression is that some of the coefficients 
# can be zero i.e. some of the features are completely neglected

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.3,random_state=30)
lasso = Lasso()
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)
print ("training score:", train_score )
print ("test score: ", test_score)
print ("number of features used: ", coeff_used)


In [ ]:
len(lasso.predict(X_test))

In [ ]:
predictions_df_2 = pd.DataFrame(lasso.predict(X_test))
predictions_df_2